# CIFAR-10 Classification using Fully Connected Neural Network (FCN)

This notebook implements a Fully Connected Neural Network (FCN) using PyTorch to classify images from the CIFAR-10 dataset. This is intended to be a baseline for comparison with a Convolutional Neural Network (CNN) model.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


## Data Preparation

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170M/170M [00:03<00:00, 48.0MB/s]


## Define the Fully Connected Neural Network

In [3]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32)  # Flatten the image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = FCN().to(device)


## Define Loss Function and Optimizer

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


## Train the Network

In [5]:
writer = SummaryWriter()
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            writer.add_scalar("training loss", running_loss / 100, epoch * len(trainloader) + i)
            running_loss = 0.0

    # Evaluate on test data after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for test_data in testloader:
            test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
            test_outputs = net(test_images)
            _, predicted = torch.max(test_outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1} Test Accuracy: {accuracy:.2f}%")
    writer.add_scalar("Test Accuracy", accuracy, epoch)
writer.close()
print("Finished Training")


[1, 100] loss: 1.862
[1, 200] loss: 1.686
[1, 300] loss: 1.635
[1, 400] loss: 1.570
[1, 500] loss: 1.545
Epoch 1 Test Accuracy: 46.55%
[2, 100] loss: 1.448
[2, 200] loss: 1.447
[2, 300] loss: 1.435
[2, 400] loss: 1.431
[2, 500] loss: 1.424
Epoch 2 Test Accuracy: 50.27%
[3, 100] loss: 1.300
[3, 200] loss: 1.302
[3, 300] loss: 1.308
[3, 400] loss: 1.321
[3, 500] loss: 1.321
Epoch 3 Test Accuracy: 51.40%
[4, 100] loss: 1.192
[4, 200] loss: 1.209
[4, 300] loss: 1.242
[4, 400] loss: 1.218
[4, 500] loss: 1.210
Epoch 4 Test Accuracy: 52.15%
[5, 100] loss: 1.107
[5, 200] loss: 1.098
[5, 300] loss: 1.121
[5, 400] loss: 1.125
[5, 500] loss: 1.143
Epoch 5 Test Accuracy: 53.69%
[6, 100] loss: 1.006
[6, 200] loss: 1.024
[6, 300] loss: 1.027
[6, 400] loss: 1.043
[6, 500] loss: 1.065
Epoch 6 Test Accuracy: 53.33%
[7, 100] loss: 0.919
[7, 200] loss: 0.930
[7, 300] loss: 0.940
[7, 400] loss: 0.959
[7, 500] loss: 0.973
Epoch 7 Test Accuracy: 54.15%
[8, 100] loss: 0.828
[8, 200] loss: 0.844
[8, 300] loss

## Evaluate on Test Data

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%")


Accuracy of the network on the 10000 test images: 54.14%
